In [1]:
Project = '/Users/daehwa/Library/CloudStorage/OneDrive-Personal/Junlab/Projects/Adipocyte'
from IPython.display import display
import pandas as pd
import gseapy as gp
import numpy as np
import pickle

gp.__version__

'1.0.6'

In [2]:

##
RD = pd.read_csv(f'{Project}/Results/Tables/CPM_RD_calculation/v20221217/adi_RD_avg_log2FC.tsv', sep='\t').rename(columns={'D4':'RD4', 'D8':'RD8'})

logFC = {}
logFC['RD'] = RD

##
APEX = pd.read_excel(f'{Project}/Results/Other_papers/Fazal_Han_etal.,2019,cell/Tables/Resources/(Sup. table4) Atlas of subcellular RNA localization revealed by APEX-seq_Cell_2019.xlsx', sheet_name='OMM APEX-seq data')
CHIP = pd.read_csv('/Users/daehwa/Library/CloudStorage/OneDrive-Personal/Junlab/Data_Library/GSEA/chip/Mouse_Ensembl_Gene_ID_Human_Orthologs_MSigDB.v2023.2.Hs.chip', sep='\t', usecols=['Probe Set ID','Gene Symbol']).rename(columns={'Probe Set ID':'gene_id', 'Gene Symbol':'Common_Gene'})

APEX2 = pd.merge(CHIP, APEX[['Common_Gene','Basal_L2FC','Puromycin_L2FC','Cycloheximide_L2FC','Mito_TargetP_score']], on='Common_Gene').sort_values('gene_id').drop_duplicates(subset='gene_id')

APEX3 = pd.merge(APEX2[['gene_id','Basal_L2FC','Puromycin_L2FC','Cycloheximide_L2FC','Mito_TargetP_score']], logFC['RD'], on='gene_id')

# filt mtDNA genes
MitoCarta3_file = '/Users/daehwa/Library/CloudStorage/OneDrive-Personal/Junlab/Data_Library/MITOCARTA3.0/Mouse.MitoCarta3.0.xls'
MitoCarta = pd.read_excel(MitoCarta3_file, 'A Mouse MitoCarta3.0')
MitoPW    = pd.read_excel(MitoCarta3_file, 'C MitoPathways')

name2ID = MitoCarta[['Symbol','EnsemblGeneID']].set_index('Symbol')['EnsemblGeneID'].to_dict()
mito_genes = MitoCarta[['EnsemblGeneID']].drop_duplicates().rename(columns={'EnsemblGeneID':'gene_id'})

MitoPW['PW level'] = MitoPW['MitoPathway Hierarchy'].str.split('>').str.len()
MitoPW['gene_ids'] = ''
for i in range(len(MitoPW)):
    for gene in MitoPW.iloc[i,:]['Genes'].split(', '):
        if type(name2ID[gene]) == float : continue
        MitoPW['gene_ids'][i] += name2ID[gene].replace('|',',') + ','
    MitoPW['gene_ids'][i] = MitoPW['gene_ids'][i][:-1]
MitoPW = MitoPW.set_index('MitoPathway',drop=True)

mtDNA = MitoCarta[['Symbol','EnsemblGeneID']]
mtDNA = mtDNA[mtDNA['Symbol'].str.startswith('mt-')].reset_index(drop=True)
mtDNA = mtDNA.rename(columns={'Symbol':'gene_name', 'EnsemblGeneID':'gene_id'})[['gene_id']]

tmp = pd.merge(mito_genes, mtDNA, on='gene_id', how='outer', indicator=True).sort_values('_merge')
mito_genes_nc = tmp[tmp['_merge']=='left_only'].drop(columns='_merge')[['gene_id']]

tmp = pd.merge(APEX3, mtDNA, on='gene_id', how='outer', indicator=True).sort_values('_merge')
APEX3 = tmp[tmp['_merge']=='left_only'].drop(columns='_merge')

APEX3_cyto = pd.merge(APEX3, mito_genes_nc, on='gene_id', how='left', indicator=True)
APEX3_cyto = APEX3_cyto[APEX3_cyto['_merge']=='left_only'].drop(columns='_merge')

APEX3_mito = pd.merge(APEX3, mito_genes_nc, on='gene_id')

# display(APEX3, APEX3_cyto, APEX3_mito)
APEX3 = APEX3.sample(frac=1).reset_index(drop=True)
APEX3_cyto = APEX3_cyto.sample(frac=1).reset_index(drop=True)
APEX3_mito = APEX3_mito.sample(frac=1).reset_index(drop=True)

APEX3_mito['Post-init'] = APEX3_mito['Cycloheximide_L2FC']-APEX3_mito['Puromycin_L2FC']

display(APEX3_mito)

/var/folders/ct/z73cq4_j2g5612tltg_pmxxc0000gn/T/ipykernel_99135/1937983951.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MitoPW['gene_ids'][i] += name2ID[gene].replace('|',',') + ','
/var/folders/ct/z73cq4_j2g5612tltg_pmxxc0000gn/T/ipykernel_99135/1937983951.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MitoPW['gene_ids'][i] = MitoPW['gene_ids'][i][:-1]


,gene_id,Basal_L2FC,Puromycin_L2FC,Cycloheximide_L2FC,Mito_TargetP_score,gene_name,RD4,RD8,Post-init
0,ENSMUSG00000004610,0.172060,0.394754,0.501723,1.0,Etfb,-0.510055,-0.754061,0.106969
1,ENSMUSG00000052738,0.397798,0.816989,0.362190,1.0,Suclg1,-0.142135,-0.117711,-0.454799
2,ENSMUSG00000028959,-0.113037,0.236100,0.513938,0.0,Fastk,-0.099066,-0.266143,0.277838
3,ENSMUSG00000024521,-0.006939,-0.008913,-0.055495,0.0,Pmaip1,0.275798,0.504255,-0.046581
4,ENSMUSG00000026618,3.803577,0.081617,5.188976,5.0,Iars2,0.029945,0.292534,5.107359
...,...,...,...,...,...,...,...,...,...
778,ENSMUSG00000030706,0.191132,0.580547,-0.183006,0.0,Mrpl48,-0.227354,-0.165974,-0.763553
779,ENSMUSG00000030802,-0.006702,0.145352,2.307026,4.0,Bckdk,0.076094,0.420585,2.161675
780,ENSMUSG00000020268,-0.194344,-0.246781,-0.455046,4.0,Lyrm7,0.086306,0.056225,-0.208265
781,ENSMUSG00000024158,0.481418,0.749426,0.909822,5.0,Hagh,-0.383863,-0.394963,0.160396


In [3]:
genelists = {
            #  'Q1_Q2' : APEX3_mito[APEX3_mito['Post-init']>1.5][['gene_id']],
            #  'Q2' : APEX3_mito[(APEX3_mito['Post-init']>1.5) & (APEX3_mito['Puromycin_L2FC']<0.4)][['gene_id']],
            #  'Q2_Q3' : APEX3_mito[APEX3_mito['Puromycin_L2FC']<0.4][['gene_id']],
             'APEXseq_Q4' : APEX3_mito[(APEX3_mito['Post-init']<1.5) & (APEX3_mito['Puromycin_L2FC']>0.4)][['gene_id']],
             }

display(genelists)

{'APEXseq_Q4':                 gene_id
 1    ENSMUSG00000052738
 6    ENSMUSG00000030335
 8    ENSMUSG00000051671
 9    ENSMUSG00000038412
 14   ENSMUSG00000044881
 ..                  ...
 775  ENSMUSG00000030614
 777  ENSMUSG00000070283
 778  ENSMUSG00000030706
 781  ENSMUSG00000024158
 782  ENSMUSG00000025465
 
 [284 rows x 1 columns]}

In [4]:
conv = pd.read_csv('/Users/daehwa/Library/CloudStorage/OneDrive-Personal/Junlab/Data_Library/GSEA/chip/Mouse_Ensembl_Gene_ID_MSigDB.v2023.2.Mm.chip', sep='\t')
conv = conv.drop(columns='Gene Title').rename(columns={'Probe Set ID':'gene_id', 'Gene Symbol':'external_gene_name'})
# display(conv)

BG_genes = APEX3_mito[['gene_id']]
BG_Symbols = pd.merge(BG_genes, conv, on='gene_id')
BG_Symbols = BG_Symbols.to_dict('list')['external_gene_name']
display(BG_Symbols[:10])
print(len(BG_genes))

['Etfb',
 'Suclg1',
 'Fastk',
 'Pmaip1',
 'Iars2',
 'Ccdc127',
 'Mrpl51',
 'Mrpl28',
 'Coa6',
 'Higd1a']

783


In [6]:
# EnrichR
results = {}
for group in genelists:
    gene_symbols = pd.merge(genelists[group], conv, on='gene_id')[['external_gene_name']]

    enr = gp.enrichr(   gene_list=gene_symbols,
                        # gene_sets='/Users/daehwa/Library/CloudStorage/OneDrive-Personal/Junlab/Data_Library/MITOCARTA3.0/MITOCARTA3_MTPW_genename.gmt',
                        gene_sets='/Users/daehwa/Library/CloudStorage/OneDrive-Personal/Junlab/Data_Library/MITOCARTA3.0/MITOCARTA3_MTPWnoChild_genename.gmt',
                        # gene_sets='/Users/daehwa/Library/CloudStorage/OneDrive-Personal/Junlab/Data_Library/GSEA/gmt/m5.go.v2023.1.Mm.symbols.gmt',
                        # gene_sets='/Users/daehwa/Library/CloudStorage/OneDrive-Personal/Junlab/Data_Library/GSEA/gmt/m5.go.cc.v2023.1.Mm.symbols.gmt',
                        background=BG_Symbols,
                        outdir=None, 
                        verbose=False
                    )
    tmp = enr.results.sort_values('Adjusted P-value')
    tmp.to_csv(f'{group}_Enrich.tsv', sep='\t', index=False)

    results[group] = tmp

    print(group)
    display(tmp[:20])

APEXseq_Q4


,Gene_set,Term,Overlap,P-value,Adjusted P-value,Odds Ratio,Combined Score,Genes
55,MITOCARTA3_MTPWnoChild_genename.gmt,OXPHOS subunits,46/60,3.628878e-11,3.193413e-09,2.331566,56.049712,Ndufb10;Ndufb7;Ndufa13;Cox6c;Sdhb;Ndufa6;Ndufs...
48,MITOCARTA3_MTPWnoChild_genename.gmt,Mitochondrial ribosome,53/76,4.433454e-10,1.950720e-08,2.137311,46.030574,Mrpl34;Mrpl21;Mrps6;Mrpl13;Mrps18b;Gadd45gip1;...
5,MITOCARTA3_MTPWnoChild_genename.gmt,CI subunits,26/34,1.235894e-06,3.625288e-05,2.227091,30.296708,Ndufb10;Ndufb7;Ndufa13;Ndufa6;Ndufs5;Ndufb11;N...
10,MITOCARTA3_MTPWnoChild_genename.gmt,CIV subunits,12/12,4.458087e-06,9.807792e-05,2.831193,34.882532,Cox7b;Cox8a;Cox5a;Cox7a2;Cox6c;Cox4i1;Cox6b1;C...
54,MITOCARTA3_MTPWnoChild_genename.gmt,OXPHOS assembly factors,24/49,4.105660e-02,6.951296e-01,1.395549,4.455712,Tmem70;Uqcc2;Ndufaf4;Cmc1;Timmdc1;Coa4;Tmem126...
63,MITOCARTA3_MTPWnoChild_genename.gmt,Respirasome assembly,3/3,4.739520e-02,6.951296e-01,2.772647,8.454449,Higd2a;Higd1a;Cox7a2l
8,MITOCARTA3_MTPWnoChild_genename.gmt,CIII subunits,6/9,6.230448e-02,7.832563e-01,1.902769,5.281557,Uqcr11;Cyc1;Uqcrfs1;Uqcrh;Uqcrq;Uqcr10
62,MITOCARTA3_MTPWnoChild_genename.gmt,ROS and glutathione metabolism,8/23,6.378397e-01,9.968845e-01,0.994844,0.447350,Hagh;Prdx4;Gpx4;Prdx5;Sod1;Prdx3;Gpx1;Msrb2
61,MITOCARTA3_MTPWnoChild_genename.gmt,"Q-linked reactions, other",1/5,8.956430e-01,9.968845e-01,0.748918,0.082541,Etfa
60,MITOCARTA3_MTPWnoChild_genename.gmt,Pyruvate metabolism,3/12,8.708325e-01,9.968845e-01,0.767389,0.106134,Mpc1;Pdhb;Mpc2
